In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Flatten, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.callbacks import ModelCheckpoint
from numpy import array, argmax
from numpy.random import rand, shuffle
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
import scipy
import statsmodels
import sklearn
import tensorflow
import keras
from io import open
import unicodedata
import random
import math
import os
import time
import re
import sys
from unicodedata import normalize
from numpy import array
from pickle import dump, load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint

INDIC_NLP_LIB_HOME=r"/content/drive/MyDrive/anoopkunchukuttan-indic_nlp_library-eccde81/src"
INDIC_NLP_RESOURCES=r"/content/drive/MyDrive/indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [5]:
def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

def to_pairs(hindi_text, telugu_text):
    hindi_lines = hindi_text.strip().split('\n')
    telugu_lines = telugu_text.strip().split('\n')
    pairs = []
    for i in range(len(telugu_lines)):
        pairs.append([])
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
        pairs[i].append(pre_process_telugu_sentence(telugu_lines[i]))
    return pairs

def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text= re.sub("'", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

def pre_process_hindi_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi",remove_nuktas)
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    line = 'START_ '+ line + ' _END'
    return (line)

def pre_process_telugu_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi",remove_nuktas)
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    line = 'START_ '+ line + ' _END'
    return (line)

hindi_text = load_doc('/content/drive/MyDrive/surya_hindi_telugu/HIN.txt')
telugu_text = load_doc('/content/drive/MyDrive/surya_hindi_telugu/TEL.txt')
pairs = to_pairs(hindi_text, telugu_text)
df = pd.DataFrame(pairs)
df.columns = ["hin", "tel"]
#df

In [6]:
lines = df

In [7]:
all_hin_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hin_words:
            all_hin_words.add(word)


all_telugu_words=set()
for tel in lines.tel:
    for word in tel.split():
        if word not in all_telugu_words:
            all_telugu_words.add(word)

#print (all_hin_words)
print (len(all_hin_words))
print (len(all_telugu_words))

12560
16242


In [8]:
# Max Length of source sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print (max_length_src)

# Max Length of target sequence
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print (max_length_tar)

85
59


In [9]:
input_words = sorted(list(all_hin_words))
target_words = sorted(list(all_telugu_words))
num_encoder_tokens = len(all_hin_words)
num_decoder_tokens = len(all_telugu_words)
num_encoder_tokens, num_decoder_tokens
num_encoder_tokens += 1
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

16243

In [10]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [11]:
# Train - Test Split
X, y = lines.hin, lines.tel
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((9899,), (1100,))

In [12]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [13]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        # range(start, stop, step)
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [14]:
latent_dim = 500
vec_len = 300 # Length of the vector that we willl get from the embedding layer
dropout_rate = 0.2 # Rate of the dropout layers
encoder_inputs = Input(shape=(None,))

#enc_emb =  Embedding(num_encoder_tokens, vec_len, mask_zero = True)(encoder_inputs)
#enc_emb =  Embedding(num_encoder_tokens, vec_len)(encoder_inputs)
enc_emb = Embedding(input_dim = num_encoder_tokens, output_dim = vec_len, mask_zero = True)(encoder_inputs)

encoder_dropout = (TimeDistributed(Dropout(rate = dropout_rate)))(enc_emb)
encoder_lstm1 = Bidirectional(LSTM(latent_dim, return_sequences=True), merge_mode='sum')(encoder_dropout)
encoder_lstm2 = Bidirectional(LSTM(latent_dim, return_sequences=True), merge_mode='sum')(encoder_lstm1)
encoder_lstm3 = Bidirectional(LSTM(latent_dim, return_sequences=True), merge_mode='sum')(encoder_lstm2)
encoder_LSTM4_layer = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM4_layer(encoder_lstm3)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [15]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, vec_len, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_dropout = (TimeDistributed(Dropout(rate = dropout_rate)))(dec_emb)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.


decoder_LSTM_layer = LSTM(latent_dim, return_sequences=True)
decoder_LSTM = decoder_LSTM_layer(decoder_dropout, initial_state = encoder_states)

decoder_LSTM_2_layer = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_LSTM_2_layer(decoder_LSTM)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.summary()

# Define a checkpoint callback :
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    3768300     input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed (TimeDistribut (None, None, 300)    0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 500)    3204000     time_distributed[0][0]           
______________________________________________________________________________________________

In [18]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 80
epochs = 10

# Compile the model

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
123/123 [==============================] - 2949s 24s/step - loss: 1.6337 - acc: 0.0827 - val_loss: 1.5019 - val_acc: 0.1083
Epoch 2/10
123/123 [==============================] - 2911s 24s/step - loss: 1.4556 - acc: 0.1059 - val_loss: 1.4934 - val_acc: 0.1099
Epoch 3/10
123/123 [==============================] - 2915s 24s/step - loss: 1.4294 - acc: 0.1035 - val_loss: 1.4734 - val_acc: 0.1127
Epoch 4/10
123/123 [==============================] - 2916s 24s/step - loss: 1.3739 - acc: 0.1120 - val_loss: 1.4643 - val_acc: 0.1173
Epoch 5/10
123/123 [==============================] - 2931s 24s/step - loss: 1.3299 - acc: 0.1172 - val_loss: 1.4757 - val_acc: 0.1188
Epoch 6/10
123/123 [==============================] - 2917s 24s/step - loss: 1.2894 - acc: 0.1213 - val_loss: 1.4725 - val_acc: 0.1201
Epoch 7/10
123/123 [==============================] - 2928s 24s/step - loss: 1.2588 - acc: 0.1268 - val_loss: 1.4645 - val_acc: 0.1218
Epoch 8/10
123/123 [==============================] - 2

In [19]:
#save the model
model.save_weights('nmt_weights.h5')

In [20]:
#load the model
model.load_weights('nmt_weights.h5')

In [21]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence
decoder_dropout2 = (TimeDistributed(Dropout(rate = dropout_rate)))(dec_emb2)
decoder_LSTM2 = decoder_LSTM_layer(decoder_dropout2, initial_state = decoder_states_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_LSTM_2_layer(decoder_LSTM2)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) #A dense softmax layer to generate prob dist. over the target vocabulary


# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 98):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [23]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [24]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ चीनी का पाचन होते समय उत्पन्न होने वाले एसिड के कारण ऐसा होता है _END
Actual telugu Translation:  చక్కెర జీర్ణక్రియ సమయంలో ఉత్పత్తి అయ్యే ఆమ్లం వల్ల ఇది జరుగుతుంది 
Predicted telugu Translation:  ఈ వ్యాధి ఎ 


In [29]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input hindi sentence:', X_train[k:k+1].values[0])
print('Actual telugu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted telugu Translation:', decoded_sentence[:-4])

Input hindi sentence: START_ वर्ष के बच्चे आईयू विटामिन ए की एक खुराक हर महीने पर देना _END
Actual telugu Translation:  సంవత్సరాల పిల్లలు ప్రతి నెలలకు మోతాదు 
Predicted telugu Translation:  ఈ వ్యాధి ఎ 
